### Extracting images of pieces from chessboards

In [41]:
import cv2 as cv
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import pickle

size = 160             #choosing the size of the image
box_size = int(size/8) #size of each cell

In [42]:
#order of pieces except pawn
pieces = ['rook', 'knight', 'bishop', 'queen', 'king', 'bishop', 'knight', 'rook']
 
#iterate over every chessboard
for file in os.listdir('./Chessboards'):
    
    img = cv.imread('./Chessboards/' + file)       #extract chessboard image
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)     #grayscale image
    gray = cv.resize(gray, (size, size))           #resizing the image

    #segragating each box image for classification
    for x in range(0, 8):
        for y in range(0, 8):
            #destination of cropped image
            directory = './cropped/'
            if x==0:
                directory = directory + 'b_' + pieces[y]
            elif x==1:
                directory = directory + 'b_pawn'
            elif x==6:
                directory = directory + 'w_pawn'
            elif x==7:
                directory = directory + 'w_' + pieces[y]
            else:
                directory = directory + 'empty'
            directory = directory + '/'
            
            #extract a single cell from the board
            crop = gray[x*box_size:x*box_size+box_size, y*box_size:y*box_size+box_size]
            
            #save the cell
            filename = directory + file + '_' + str(x) + 'x' + str(y) + '.jpg'
            if not os.path.exists(filename):
                cv.imwrite(filename, crop)

### Creating the training data

In [43]:
DATADIR = "./cropped"
CATEGORIES = ['b_rook', 'b_knight', 'b_queen', 'w_king', 'b_king', 'w_queen', 'w_bishop', 'w_knight', 'b_bishop', 'w_rook', 'empty', 'b_pawn', 'w_pawn']

training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array = cv.imread(os.path.join(path,img), cv.IMREAD_GRAYSCALE)
            training_data.append([img_array, class_num])
            
create_training_data()        #create training data
random.shuffle(training_data) #shuffling data to improve accuracy


In [44]:
print(len(training_data))

896


In [47]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

#Converting the list X to numpy array as we cannot feed a list to neural network
X = np.array(X).reshape(-1, box_size, box_size, 1)
X = X/255.0

### Storing data in pickle

In [48]:
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()